In [ ]:
import numpy as np

In [ ]:
#２層フィードフォワードニューラルネットワークモデル
def FNN(x,w,v,M,K):
    #x:入力データ  (N*D)→入力データはNセット分、まとめて一度に処理させる。入力データ１セットに対し出力値１セットを得る。
    #w:中間層への重み行列 (M*(D+1))
    #v:出力層への重み行列 (K*(M+1))
    #M:中間層の次元
    #K:出力層の次元
    N,D = x.shape
    b=np.zeros((N,M+1))  #中間層の入力総和
    z=np.zeros((N,M+1))  #中間層の出力
    a=np.zeros((N,K))    #出力層の入力総和
    y=np.zeros((N,K))    #出力層の出力
    
    #(1~n)個目の入力データセットについての処理
    for n in range(N):
        #中間層の計算
        for m in range(M):
            #入力データxを利用し、入力総和bのm番目の要素の算出を行う。結果はb[n,m]
            b[n,m]=np.dot(w[m,:],np.r_[x[n,:],1])
            #入力総和をシグモイド関数で出力値にする
            z[n,m]=sigmoid(b[n,m]) 

        #出力層の計算
        z[n,M] = 1  #最後のダミー用データ
        u=0
        for k in range(K):
            #中間層からの入力データzを利用し、入力総和aのk番目の要素の算出を行う。結果はa[n,k]
            a[n,k]=np.dot(v[k,:],z[n,:]) 
            #ソフトマックス関数用の入力総和
            u+=np.exp(a[n,k])
        for k in range(K):
            y[n,k]=np.exp(a[n,k])/u
    return y,a,z,b
            

In [ ]:
#平均交差エントロピー誤差
def E_FNN(w,v,M,K,x,t):
    N,D=x.shape
    y,a,z,b=FNN(x,w,v,M,K)
    e = -np.dot(np.log(y.reshape(-1)),t.reshape(-1)) / N
    return e

In [ ]:
#数値微分
def dE_FNN(w,v,M,K,x,t):
    epsilon = 0.001
    dw = np.zeros_like(w)
    dv = np.zeros_like(v)
    
    e=E_FNN(w,v,M,K,x,t)
    for i in range(len(w)):
        for j in range(len(w[i])):
            w_ep = w.copy()
            w_ep[i][j]=w_ep[i][j]+epsilon
            dw_epij=E_FNN(w_ep,v,M,K,x,t)
            dw[i][j]=(dw_epij-e)/epsilon
    
    for i in range(len(v)):
        for j in range(len(v[i])):
            v_ep = v.copy()
            v_ep[i][j]=v_ep[i][j]+epsilon
            dv_epij=E_FNN(w,v_ep,M,K,x,t)
            dv[i][j]=(dv_epij-e)/epsilon

    return dw,dv
    

In [ ]:
#数値微分を使った勾配法